In [6]:
import pickle

from util import read_bills
from regex import regex
from http.client import HTTPConnection

In [58]:
bills = read_bills('../lab2/data_sample')

In [59]:
connection = HTTPConnection('127.0.0.1:9200')
headers = {'Content-type': 'text/plain'}

In [60]:
def request_tagging(bill_content):
    body = bill_content.encode('utf-8')
    connection.request('POST', '/', body, headers)
    response = connection.getresponse()
    return response.read().decode('utf-8')


def tagged_to_tokens(tags):
    pattern = r'(?=^(\p{L}+)).*?\n\s(\p{L}*)\s(\p{L}*)'
    results = list(regex.finditer(pattern, tags, flags=regex.MULTILINE))
    return ['{0}:{1}'.format(r.group(2), r.group(3)) for r in results]

---

In [7]:
from lab4.ngrams import calculate_counters, calculate_llr, highest_scores

In [62]:
tokens = []

for _, content in bills.items():
    tags = request_tagging(content)
    for token in tagged_to_tokens(tags):
        tokens.append(token)

In [8]:
# pickle.dump(tokens, open('tokens.p', 'wb'))

tokens = pickle.load(open('tokens.p', 'rb'))

In [9]:
left, right, bigram = calculate_counters(tokens, [])
llr = calculate_llr(left, right, bigram)

In [10]:
filtered = {(a,b): score for (a, b), score in llr.items() if a.endswith('subst') and (b.endswith('subst') or b.endswith('adj'))}

In [11]:
for (a, b), score in highest_scores(filtered, 50):
    bigram = '{0} {1}'.format(a, b)
    print('{0:40}: {1}'.format(bigram, score))

minister:subst właściwy:adj             : 44373.011963426485
droga:subst rozporządzenie:subst        : 35983.374974073435
Rzeczpospolita:subst polski:adj         : 26153.50254162657
samorząd:subst terytorialny:adj         : 17227.503198456485
jednostka:subst organizacyjny:adj       : 16959.22027067613
produkt:subst leczniczy:adj             : 15686.28551903789
skarb:subst państwo:subst               : 15037.56413960451
papier:subst wartościowy:adj            : 14130.505112213665
unia:subst europejski:adj               : 12683.297045598316
ubezpieczenie:subst społeczny:adj       : 12064.809143047634
działalność:subst gospodarczy:adj       : 11405.904457978788
straż:subst graniczny:adj               : 10824.018597995833
dzień:subst grudzień:subst              : 10758.916179734166
terytorium:subst Rzeczpospolita:subst   : 10693.877965768479
finanse:subst publiczny:adj             : 10560.51819419983
obrona:subst narodowy:adj               : 10162.701145730185
dzień:subst styczeń:subst    

---

Using stopwords:

In [12]:
def read_stopwords(path):
    with open(path, 'r') as f:
        return [regex.sub(r'\s', '', word) for word in f.readlines()]

In [13]:
polish_stopwords = read_stopwords('../lab4/data/stopwords-pl.txt')

In [14]:
left, right, bigram = calculate_counters(tokens, polish_stopwords)
llr = calculate_llr(left, right, bigram)

In [15]:
filtered = {(a,b): score for (a, b), score in llr.items() if a.endswith('subst') and (b.endswith('subst') or b.endswith('adj'))}

In [16]:
for (a, b), score in highest_scores(filtered, 50):
    bigram = '{0} {1}'.format(a, b)
    print('{0:40}: {1}'.format(bigram, score))

minister:subst właściwy:adj             : 34175.70605477318
droga:subst rozporządzenie:subst        : 29494.761500026914
Rzeczpospolita:subst polski:adj         : 21808.199546717748
samorząd:subst terytorialny:adj         : 14766.68815404427
jednostka:subst organizacyjny:adj       : 13946.958468182478
produkt:subst leczniczy:adj             : 13393.99915473722
skarb:subst państwo:subst               : 12698.039478285296
papier:subst wartościowy:adj            : 12443.749091597543
unia:subst europejski:adj               : 11024.825482128828
ubezpieczenie:subst społeczny:adj       : 10040.660229359462
działalność:subst gospodarczy:adj       : 9407.35585392773
straż:subst graniczny:adj               : 9324.93175425155
terytorium:subst Rzeczpospolita:subst   : 9113.304772837088
finanse:subst publiczny:adj             : 8810.905567078444
obrona:subst narodowy:adj               : 8734.162767812115
państwo:subst członkowski:adj           : 8589.345137608994
administracja:subst rządowy:adj    